In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from pymote import Node
from pymote import Network

# Reseting numbering
Node.cid = 1

# Creating network
net = Network()
"""
# Adding nodes
n1 = net.add_node(pos=(100, 300), commRange=0)
n2 = net.add_node(pos=(200, 200), commRange=0)
n3 = net.add_node(pos=(200, 400), commRange=0)
n4 = net.add_node(pos=(400, 200), commRange=0)
n5 = net.add_node(pos=(400, 400), commRange=0)
n6 = net.add_node(pos=(500, 300), commRange=0)
n7 = net.add_node(pos=(550, 450), commRange=0)

# Adding weighted edges
net.add_weighted_edges_from([
    (n1, n2, 10),
    (n1, n3, 1),
    (n2, n4, 2),
    (n3, n5, 3),
    (n1, n6, 5),
    (n4, n6, 8),
    (n5, n6, 3),
    (n5, n7, 5),
    (n6, n7, 3)
])
"""

n1 = net.add_node(pos=(100, 300), commRange=0)
n2 = net.add_node(pos=(200, 300), commRange=0)
n3 = net.add_node(pos=(150, 400), commRange=0)

net.add_edges_from([
    (n1, n2),
    (n1, n3),
    (n2, n3),
])
net.show()

In [ ]:
# setup sensors
import scipy.stats
from pymote.sensor import DistSensor

trueDistSensor = DistSensor({'pf': scipy.stats.norm, 'scale': 0 }) # no measurement noise
for n in net.nodes():
    n.compositeSensor = ('NeighborsSensor', trueDistSensor)

In [ ]:
from pymote import NetworkGenerator
from pymote import write_pickle
from pymote import Simulation
from pymote.algorithms.localization import IterativeLocalization

net.algorithms = (IterativeLocalization, )

write_pickle(net, "test-net.npc.gz")

sim = Simulation(net)
sim.run()

for n in net.nodes():
    print(n.id, n.status)

In [ ]:
print("Edges:")
for e in net.edges(data=True):
    print(e)
    
print("Nodes:")
for n in net.nodes():
    print("dists:", n.memory['neighborDistances'])
    print(n.id, n.memory['position'])

In [ ]:
# show localized network
import numpy as np

#print({n: n.memory['position'] for n in net.nodes()})
#net.show(positions={n: np.array(n.memory['position']) for n in net.nodes()})

for node in net.nodes():
    print("mem dists:", node.memory['neighborDistances'])
    
positions = [np.array(n.memory['position']) for n in net.nodes()]
pairs = [
    (positions[0], positions[1]),
    (positions[1], positions[2]), 
    (positions[0], positions[2]), 
]
for (i,j) in pairs:
    print("localized dist:", np.sqrt(np.sum((i - j) ** 2)))
net.show()
net.pos = {n: np.array(n.memory['position']) for n in net.nodes()}
net.show()